In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2 as cv
import pandas as pd
import os
import pathlib
import sys
import numpy as np
import random
import shutil
import cv2 as cv
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
# from tensorflow.keras.utils import np_utils
from sklearn import metrics
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

Download the dataset from: 

https://www.aicrowd.com/challenges/snakeclef2021-snake-species-identification-challenge/dataset_files

Files to be download:
1 - SnakeCLEF-2021 - TrainingData
2 - SnakeCLEF2021 - TrainVal Metadata

List and id of species for this projects
-------------------------------
001 |Vipera berus
002 |Vipera ursinii
003 |Vipera seoanei
004 |Vipera ammodytes
005 |Vipera seoanei
006 |Vipera xanthina
007 |Vipera nikolskii

To do List
1 - Unzip the Train Data
2 - Found the path to each species on TrainVal MetadatA
3 - Create one folder for each image on our list(folder nome the ID)
4 - Randonaly split the dataset
Dataset Clean
1 - Remove bad images (visual inspection)
2 - Normalize all images of dataset (same size and dived by 256)

In [13]:
pathtxt = pathlib.Path('pathtxt/')

data = pathlib.Path('datasetproject/')
train_data = pathlib.Path('datasetproject/train')
test_data = pathlib.Path('datasetproject/test')

if not os.path.exists(data):
    os.makedirs(data)
if not os.path.exists(train_data):
    os.makedirs(train_data)
if not os.path.exists(test_data):
    os.makedirs(test_data)

def creat_fold():

    for filename in pathtxt.glob('*.txt'):
        foldername = filename.name.split('.')[0]
        print(foldername)
        pathSpecies =train_data.joinpath(foldername)
        pathlib.Path(pathSpecies).mkdir()

        with open(filename) as datafile:
            for i, line in enumerate(datafile.readlines()):
                line = line.rstrip('\n')
                name = line.split('/')[-1]
                shutil.copyfile(line, train_data.joinpath(foldername).joinpath(name))

# creat_fold()


In [14]:

split_ratio = 0.20

def split_data():

    for dr in train_data.glob('*/'):
        pathSpecies =test_data.joinpath(dr.name)
        pathlib.Path(pathSpecies).mkdir()
        files = os.listdir(dr)

        random.shuffle(files)

        qtd = round(len(files) * split_ratio)
        print(qtd)

        for i in range(qtd):
            src = train_data.joinpath(dr.name).joinpath(files[i])
            dst = test_data.joinpath(dr.name).joinpath(files[i])
            os.rename(src, dst)

    return 

# split_data()
